<a href="https://colab.research.google.com/github/JuyeongKime2/OpenAI/blob/main/OpenAI_SQL%EC%BF%BC%EB%A6%AC%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. 라이브러리 불러오기**
(주피터 실습이므로 코드는 변경)

In [ ]:
!pip install openai #패키지설치

In [ ]:
#패키지 불러오기
import pandas as pd
import os
from openai import OpenAI

In [ ]:
!pip list #설치된 패키지 확인

**2. 데이터 불러오기**

```
# 주피터로 실행 시 코드
df=pd.read_excel("AdidasSalesdata 2.xlsx")
df.head(5)
```



In [ ]:
#Google Colab에서 업로드한 .xlsx 데이터셋을 읽기위한 패키지
!pip install pandas openpyxl

In [ ]:
#.xlsx 파일을 선택하여 업로드
from google.colab import files
uploaded = files.upload()

In [ ]:
#업로드된 파일의 이름 가져오기
file_name = list(uploaded.keys())[0]

#pandas를 사용하여 엑셀 파일 읽기
df=pd.read_excel(file_name)

#데이터 프레임 출력
df.head(10) #tail(끝부터)

In [ ]:
df.columns #띄어쓰기 되어있음

In [ ]:
#공백을 언더바로 변경
df.columns = df.columns.str.replace(' ','_')
df.columns

#예시: df.columns=[col.replace(' ', '_') for col df.columns]

2.1 데이터 확인하기

> groupby를 활용해서 데이터를 확인하는 것이 기초적인 데이터 분석의 방법



In [ ]:
#각 Region별로 total sales의 sum을 출력(sum of sales by region)
#df.groupby('Region')['Total_Sales'].sum() #오름차순
#df.groupby('Region').sum()['Total_Sales']
df.groupby('Region')['Total_Sales'].sum().sort_values(ascending=False) #내림차순으로


**3. SQL Database 셋팅하기**

In [ ]:
#패키지 설치
from sqlalchemy import create_engine #임시 데이터베이스(DB) 생성 패키지
from sqlalchemy import text #쿼리생성 패키지

In [ ]:
# 임시 SQLite 데이터베이스 생성
temp_db = create_engine('sqlite:///:memory:', echo=True) #in-house db 또는 temp db

In [ ]:
#데이터프레임을 'Sales' 테이블로 저장, Connection은 temp db로 지정
data=df.to_sql(name='Sales', con=temp_db) #데이터베이스 연결 객체 생성

In [ ]:
#Temp DB를 Connect 해서 Result에 담기
with temp_db.connect() as conn:
  #Total sales가 가장 높은 Descending order로 하고 하나만 출력
    result = conn.execute(text("SELECT Retailer,City, Total_Sales,Operating_Profit from Sales ORDER BY Total_Sales DESC LIMIT 1"))

In [ ]:
result.all() #결과 출력 : 올마트 올랜도가 total sales 가장 높음

**4. OpenAI 셋팅**


4.1 API 사용을 위한 Key 입력 및 셋팅

In [ ]:
#key를 OSNR에 담기
os.environ['OPENAI_API_KEY']='MY_API_KEY'
client=OpenAI()

In [ ]:
#GPT 한테 우리가 다루는 데이터,테이블이 어떤 구조인지 알려주는 함수
#table_definition_prompt함수 정의
def table_definition_prompt(df):

##GPT 모델에게 columns 알려주기
#앞으로 생성할 쿼리는 이 테이블 구조에 바탕이 된 내용으로 작성하라는 명령
    prompt = """Given the following sqlite SQL definition,
                 write queries based on the request
                 \n### sqlite SQL table, with its properities(columns):
    #
    # Sales({})
    #
    """.format(",".join(str(x) for x in df.columns))

    return prompt


In [ ]:
print(table_definition_prompt(df))

In [ ]:
#사용자로부터 어떤걸 확인하고 싶은지 받는 내용의 함수
def prompt_input():
  nlp_text=input("질의 하고자하는 내용을 입력해주세요")
  return nlp_text

In [ ]:
nlp_text= prompt_input() #사용자로부터 받은 내용 저장

In [ ]:
#테이블 구조를 알려주는 내용+실제로 궁금한 내용= Full prompt
full_prompt = str(table_definition_prompt(df)) + str(nlp_text)  #데이터 타입을 문자열로 변환
full_prompt

**4.2 API Call**

(OpenAI API > Text genaration > Chat Completions 사용)

In [ ]:
##오류 잡아야함
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are an assistant that generates SQL queries based on the given SQLite table definition and a natural language request. The query should start with 'SELECT' and end with a semicolon(;)."},
         {"role": "user", "content": f"A query to answer: {full_prompt}"},
    ],
    #토큰의 최대 길이를 200으로 생성
    max_tokens=200,
    #생성되는 응답에 대한 창의성 (0~1 까지 지정)
    temperature= 1.0,
    #특정한 문자열이 들어오면 생성 종료
    stop= None
)
# 응답 출력
print(response)

In [ ]:
response

In [ ]:
#필요한 부분만 추출
response['choices'][0].message.content

In [ ]:
#정교한 함수 처리(핸들링 하는 함수 생성)
def handle_response(response):
    query= response['choices'][0].message.content.strip()

   #만약 쿼리가 selct로 시작을 안하면 select를 앞으로 붙여주기
    if not query.upper().startswith('SELECT'):
        query = 'SELECT ' + query
  #만약 semi-colon으로 안끝나면 붙여주기
    if not query.endswith(';'):
        query = query + ';'

    return query



In [ ]:
print(handle_response(response))

 **4.2.1 Query를 DB에 입력해보자**

In [ ]:
with temp_db.connect() as conn:
  #Temp DB를 Connect 해서 Result에 담기
    result = conn.execute(text(handle_response(response)))